In [ ]:
import numpy as np

Rax, Zax = 1.0, 0.0
Rell, Zell = 0.3, 0.5
phi0 = 0.0

BPhiax = 2.5

m = 3 # toroidal turn number
n = 1
iota = float(n)/float(m)

def cycleRZ(phi):
    return np.array([
        Rell * np.cos(iota*phi+phi0) + Rax,
        Zell * np.sin(iota*phi+phi0) + Zax
    ])

dTETdphi = 1./3.
def TETu(phi):
    return phi/3 + np.pi/2 + np.pi/9
def TETs(phi):
    return phi/3 + np.pi/2 - np.pi/9

lamu = np.e**(1/5)
lams = np.e**(-1/5)

def BR0BZ0(phi):
    Rc = cycleRZ(phi)[0]
    BR0 = - iota * Rell * np.sin(iota*phi+phi0)      /Rc*BPhi(phi, cycleRZ(phi))
    BZ0 = + iota * Zell * np.cos(iota*phi+phi0)      /Rc*BPhi(phi, cycleRZ(phi))
    return np.array([BR0, BZ0])
def BRBZ(phi, X_pol):
    
    V = np.array([
        [np.cos(TETu(phi) ), np.cos(TETs(phi) )],
        [np.sin(TETu(phi) ), np.sin(TETs(phi) )],
    ])
    Lam = np.array([
        [np.log( np.abs(lamu) )/(2*m*np.pi), 0],
        [0, np.log( np.abs(lams) )/(2*m*np.pi)],
    ])
    
    A = V @ Lam @ np.linalg.inv(V) + np.array([
        [0.0, -dTETdphi],
        [dTETdphi, 0.0],
    ]) 
    A[:,0]-= 2 * BR0BZ0(phi) / BPhi(phi, cycleRZ(phi) )
    Rc = cycleRZ(phi)[0]
    return BR0BZ0(phi)  +  BPhi(phi, cycleRZ(phi) ) /Rc * A @ (X_pol-cycleRZ(phi))

def BPhi(phi, X_pol):
    return Rax * BPhiax / X_pol[0]
def frac_RBpol_BPhi(phi, X_pol):
    return X_pol[0] * BRBZ(phi, X_pol) / BPhi(phi, X_pol)

def dRBpol_BPhi_dRZ(phi):
    V = np.array([
        [np.cos(TETu(phi) ), np.cos(TETs(phi) )],
        [np.sin(TETu(phi) ), np.sin(TETs(phi) )],
    ])
    Lam = np.array([
        [np.log( np.abs(lamu) )/(2*m*np.pi), 0],
        [0, np.log( np.abs(lams) )/(2*m*np.pi)],
    ])
    
    A = V @ Lam @ np.linalg.inv(V) + np.array([
        [0.0, -dTETdphi],
        [dTETdphi, 0.0],
    ]) 
    return A

In [ ]:
from scipy.integrate import solve_ivp
DXpol_ivp = solve_ivp(lambda phi, DXpol: (dRBpol_BPhi_dRZ(phi) @ DXpol.reshape(2,2) ).flatten(), [0, 2*m*np.pi], np.array([[1.0, 0.0], [0.0, 1.0]]).flatten(), dense_output=True, max_step=2*np.pi/10000)
DPm_ivp = solve_ivp(lambda phi, DPm: (
    dRBpol_BPhi_dRZ(phi) @ DPm.reshape(2,2)
-   DPm.reshape(2,2) @ dRBpol_BPhi_dRZ(phi) 
).flatten(), [0, 2*m*np.pi], DXpol_ivp.sol(2*m*np.pi), dense_output=True, max_step=2*np.pi/10000)


In [ ]:
from plotly.graph_objs import Scatter3d
from plotly.subplots import make_subplots
import numpy as np

import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=1,
    specs=[[{'is_3d': True}, ]],
)

nPhi, nS = 400, 40
Phi = np.linspace(0.0, 2*3*np.pi - 2./3*np.pi , num=nPhi, endpoint=True)
TETu = + Phi/3 + np.pi/2 + np.pi/9
TETs = + Phi/3 + np.pi/2 - np.pi/9

Rax, Zax = 1.0, 0.0
Rell, Zell = 0.3, 0.5
iota = 1./3.
phi0 = 0.0
Rc = Rell * np.cos(iota*Phi+phi0) + Rax
Zc = Zell * np.sin(iota*Phi+phi0) + Zax



## The four branches of the invariant manifolds

R_isphi = Rc + np.linspace(0.0, 0.1, num=nS)[:,None] * np.cos(TETs)[None,:] # [iS, iPhi]
Z_isphi = Zc + np.linspace(0.0, 0.1, num=nS)[:,None] * np.sin(TETs)[None,:] # [iS, iPhi]
S_isphi = np.linspace(0.0, 0.1, num=nS)[:,None] * np.ones_like(TETs)[None,:] # [iS, iPhi]
x_isphi = R_isphi*np.cos(Phi)[None,:]
y_isphi = R_isphi*np.sin(Phi)[None,:] 
z_isphi = Z_isphi

fig.add_trace(
    go.Surface(
        x=x_isphi,
        y=y_isphi,
        z=z_isphi,
        surfacecolor=S_isphi, 
        colorscale="Blues_r",
#         opacityscale=[
#             [0.0, 0.3], # vertex with min value is totally opaque  
# #             [0.1, 0.9],
# #             [0.2, 0.7],
#             [0.5, 0.8],
#             [1.0, 0.0] # vertex with max value is totally transparent  
#         ],
    ),1,1, )

R_isphi = Rc + np.linspace(0.0, 0.1, num=nS)[:,None] * np.cos(TETu)[None,:] # [iS, iPhi]
Z_isphi = Zc + np.linspace(0.0, 0.1, num=nS)[:,None] * np.sin(TETu)[None,:] # [iS, iPhi]
S_isphi = np.linspace(0.0, 0.1, num=nS)[:,None] * np.ones_like(TETu)[None,:] # [iS, iPhi]
x_isphi = R_isphi*np.cos(Phi)[None,:]
y_isphi = R_isphi*np.sin(Phi)[None,:] 
z_isphi = Z_isphi

fig.add_trace(
    go.Surface(
        x=x_isphi,
        y=y_isphi,
        z=z_isphi,
        surfacecolor=S_isphi, 
        colorscale="Oranges_r",
#         opacityscale=[
#             [0.0, 0.3], # vertex with min value is totally opaque  
# #             [0.1, 0.9],
# #             [0.2, 0.7],
#             [0.5, 0.8],
#             [1.0, 0.0] # vertex with max value is totally transparent  
#         ],
    ),1,1, )

R_isphi = Rc - np.linspace(0.0, 0.1, num=nS)[:,None] * np.cos(TETs)[None,:] # [iS, iPhi]
Z_isphi = Zc - np.linspace(0.0, 0.1, num=nS)[:,None] * np.sin(TETs)[None,:] # [iS, iPhi]
S_isphi = np.linspace(0.0, 0.1, num=nS)[:,None] * np.ones_like(TETs)[None,:] # [iS, iPhi]
x_isphi = R_isphi*np.cos(Phi)[None,:]
y_isphi = R_isphi*np.sin(Phi)[None,:] 
z_isphi = Z_isphi

fig.add_trace(
    go.Surface(
        x=x_isphi,
        y=y_isphi,
        z=z_isphi,
        surfacecolor=S_isphi, 
        colorscale="Blues_r",
#         opacityscale=[
#             [0.0, 0.3], # vertex with min value is totally opaque  
# #             [0.1, 0.9],
# #             [0.2, 0.7],
#             [0.5, 0.8],
#             [1.0, 0.0] # vertex with max value is totally transparent  
#         ],
    ),1,1, )

R_isphi = Rc - np.linspace(0.0, 0.1, num=nS)[:,None] * np.cos(TETu)[None,:] # [iS, iPhi]
Z_isphi = Zc - np.linspace(0.0, 0.1, num=nS)[:,None] * np.sin(TETu)[None,:] # [iS, iPhi]
S_isphi = np.linspace(0.0, 0.1, num=nS)[:,None] * np.ones_like(TETu)[None,:] # [iS, iPhi]
x_isphi = R_isphi*np.cos(Phi)[None,:]
y_isphi = R_isphi*np.sin(Phi)[None,:] 
z_isphi = Z_isphi

fig.add_trace(
    go.Surface(
        x=x_isphi,
        y=y_isphi,
        z=z_isphi,
        surfacecolor=S_isphi, 
        colorscale="Oranges_r",
#         opacityscale=[
#             [0.0, 0.3], # vertex with min value is totally opaque  
# #             [0.1, 0.9],
# #             [0.2, 0.7],
#             [0.5, 0.8],
#             [1.0, 0.0] # vertex with max value is totally transparent  
#         ],
    ),1,1, )


# Cycle trace
# fig.add_trace(    
#     go.Scatter3d(
#         x=Rc*np.cos(Phi), 
#         y=Rc*np.sin(Phi), 
#         z=Zc, mode="lines", line=dict(color='black', width=3) ),               
# )

# # FLT trajectory
# fig.add_trace(    
#     go.Scatter3d(
#         x=cylflt.sol(Phi*9)[0,:]*np.cos(Phi*9), 
#         y=cylflt.sol(Phi*9)[0,:]*np.sin(Phi*9), 
#         z=cylflt.sol(Phi*9)[1,:], mode="lines", line=dict(color='black', width=3) ),               
# )





# Reference transparent torus
torus_nPhi = 60
torus_Phi = np.linspace(0.0, 2*np.pi*(2./3.), num=torus_nPhi, endpoint=True)

torus_Rc = Rell * np.cos(iota* np.linspace(0.0, 2*3*np.pi) + phi0) + Rax
torus_Zc = Zell * np.sin(iota* np.linspace(0.0, 2*3*np.pi) + phi0) + Zax

x_isphi = torus_Rc[:,None] * np.cos(torus_Phi)[None,:]
y_isphi = torus_Rc[:,None] * np.sin(torus_Phi)[None,:] 
z_isphi = torus_Zc[:,None] * np.ones_like(torus_Phi)[None,:]

fig.add_trace(
    go.Surface(
        x=x_isphi,
        y=y_isphi,
        z=z_isphi,
        surfacecolor=np.abs(z_isphi),
        colorscale="greys", 
#         colorscale="Oranges_r",
#         opacityscale=[
#             [0.0, 0.8], # vertex with min value is totally opaque  
# #             [0.1, 0.9],
# #             [0.2, 0.7],
#             [0.5, 0.8],
#             [1.0, 0.0] # vertex with max value is totally transparent  
#         ],
        opacity=0.20,
    ),1,1, )




def draw_eigvec_at_Phi(fig, Phi, DPm_ivp,arrowlen=10e-2):
    import numpy.linalg as LA
    eigvals, eigvecs = LA.eig(
        np.moveaxis(DPm_ivp.sol(Phi).reshape((2,2,-1)), 2, 0)
    )
    # Exchange eigenvectors such that 
    #   eigvals[:,0] in [0,1], 
    #   eigvals[:,1] in [0,+\infinity)
    for i in range(len(Phi)):
        if eigvals[i,0] > 1.0:
#             temp = np.copy( eigvecs[i,0,:] )
            eigvecs[i,:,[0,1]] = eigvecs[i,:,[1,0]]
    
    cycleRZs = np.asarray([cycleRZ(phi) for phi in Phi])
    Rc, Zc = cycleRZs[:,0], cycleRZs[:,1]
    
    
    for i in range(len(Phi)):
        circTET = np.linspace(0.0, 2*np.pi)
        circR = Rc[i] + arrowlen * np.cos(circTET)
        circZ = Zc[i] + arrowlen * np.sin(circTET)
        fig.add_trace(    
            go.Scatter3d(
                x=circR * np.cos(Phi[i]), 
                y=circR * np.sin(Phi[i]), 
                z=circZ, mode="lines", line=dict(color='grey', width=2) ),               
        )
        
    Rend = Rc + arrowlen * eigvecs[:,0,0]
    Zend = Zc + arrowlen * eigvecs[:,1,0]
    fig.add_trace(    
        go.Cone(
            x=Rend*np.cos(Phi), #[Rc*np.cos(phi), Rend*np.cos(phi)], 
            y=Rend*np.sin(Phi), #[Rc*np.sin(phi), Rend*np.sin(phi)], 
            z=Zend, #[Zc, Zend],
            u=+Rc*np.cos(Phi) - Rend*np.cos(Phi), 
            v=+Rc*np.sin(Phi) - Rend*np.sin(Phi), 
            w=+Zc -Zend, 
            cauto=False, cmin= 0.0e-2, cmax= 5e-2,
#             lighting_ambient=0.8,
            sizemode="absolute",
            sizeref=4e-2,
            colorscale="Blues",
        ),               
    )
    Rend = Rc - arrowlen * eigvecs[:,0,0]
    Zend = Zc - arrowlen * eigvecs[:,1,0]
    fig.add_trace(    
        go.Cone(
            x=Rend*np.cos(Phi), #[Rc*np.cos(phi), Rend*np.cos(phi)], 
            y=Rend*np.sin(Phi), #[Rc*np.sin(phi), Rend*np.sin(phi)], 
            z=Zend, #[Zc, Zend],
            u=+Rc*np.cos(Phi) - Rend*np.cos(Phi), 
            v=+Rc*np.sin(Phi) - Rend*np.sin(Phi), 
            w=+Zc -Zend, 
            cauto=False, cmin= 0.0e-2, cmax= 5e-2,
#             lighting_ambient=0.8,
            sizemode="absolute",
            sizeref=4e-2,
            colorscale="Blues",
        ),               
    )
    Rend = Rc + arrowlen * eigvecs[:,0,1]
    Zend = Zc + arrowlen * eigvecs[:,1,1]
    fig.add_trace(    
        go.Cone(
            x=Rend*np.cos(Phi), #[Rc*np.cos(phi), Rend*np.cos(phi)], 
            y=Rend*np.sin(Phi), #[Rc*np.sin(phi), Rend*np.sin(phi)], 
            z=Zend, #[Zc, Zend],
            u=-Rc*np.cos(Phi) + Rend*np.cos(Phi), 
            v=-Rc*np.sin(Phi) + Rend*np.sin(Phi), 
            w=-Zc +Zend, 
            cauto=False, cmin= 0.0e-2, cmax= 10e-2,
#             lighting_ambient=0.8, 
            sizemode="absolute",
            sizeref=4e-2,
            colorscale="Reds",
        ),               
    )
    Rend = Rc - arrowlen * eigvecs[:,0,1]
    Zend = Zc - arrowlen * eigvecs[:,1,1]
    fig.add_trace(    
        go.Cone(
            x=Rend*np.cos(Phi), #[Rc*np.cos(phi), Rend*np.cos(phi)], 
            y=Rend*np.sin(Phi), #[Rc*np.sin(phi), Rend*np.sin(phi)], 
            z=Zend, #[Zc, Zend],
            u=-Rc*np.cos(Phi) + Rend*np.cos(Phi), 
            v=-Rc*np.sin(Phi) + Rend*np.sin(Phi), 
            w=-Zc +Zend, 
            cauto=False, cmin= 0.0e-2, cmax= 10e-2,
#             lighting_ambient=0.8,
            sizemode="absolute",
            sizeref=4e-2,
            colorscale="Reds",),               
    )
    


def draw_eigvec_at_phi(fig, phi, DPm_ivp, arrowlen=5e-2):
    import scipy.linalg as LA
    eigvals, eigvecs = LA.eig(DPm_ivp.sol(phi).reshape((2,2)))
    
    Rc, Zc = cycleRZ(phi)
    circTET = np.linspace(0.0, 2*np.pi)
    circR = Rc + arrowlen * np.cos(circTET)
    circZ = Zc + arrowlen * np.sin(circTET)
    fig.add_trace(    
        go.Scatter3d(
            x=circR * np.cos(phi), 
            y=circR*np.sin(phi), 
            z=circZ, mode="lines", line=dict(color='grey', width=2) ),               
    )
    for i in range(2):
        if eigvals[i] > 1.0:
            Rc, Zc = cycleRZ(phi)
            Rend, Zend = cycleRZ(phi) + arrowlen * eigvecs[:,i]
            Rarr1,Zarr1 = cycleRZ(phi) + 0.7 * arrowlen * eigvecs[:,i] + 0.3* arrowlen*np.array([[0.0, -1.0], [1.0, 0.0]])@eigvecs[:,i]
            Rarr2,Zarr2 = cycleRZ(phi) + 0.7 * arrowlen * eigvecs[:,i] - 0.3* arrowlen*np.array([[0.0, -1.0], [1.0, 0.0]])@eigvecs[:,i]
            fig.add_trace(    
                go.Cone(
                    x=[Rend*np.cos(phi)], #[Rc*np.cos(phi), Rend*np.cos(phi)], 
                    y=[Rend*np.sin(phi)], #[Rc*np.sin(phi), Rend*np.sin(phi)], 
                    z=[Zend], #[Zc, Zend],
                    u=[-Rc*np.cos(phi) + Rend*np.cos(phi)], 
                    v=[-Rc*np.sin(phi) + Rend*np.sin(phi)], 
                    w=[-Zc +Zend], 
                    colorscale="Reds"),               
                    
            )
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rc*np.cos(phi), Rend*np.cos(phi)], 
                    y=[Rc*np.sin(phi), Rend*np.sin(phi)], 
                    z=[Zc, Zend], mode="lines", line=dict(color="red", width=5), opacity=0.5 ),               
            )
#             fig.add_trace(    
#                 go.Scatter3d(
#                     x=[Rarr1*np.cos(phi), Rend*np.cos(phi)], 
#                     y=[Rarr1*np.sin(phi), Rend*np.sin(phi)], 
#                     z=[Zarr1, Zend], mode="lines", line=dict(color="red",width=5), opacity=0.5 ),               
#             )
#             fig.add_trace(    
#                 go.Scatter3d(
#                     x=[Rarr2*np.cos(phi), Rend*np.cos(phi)], 
#                     y=[Rarr2*np.sin(phi), Rend*np.sin(phi)], 
#                     z=[Zarr2, Zend], mode="lines", line=dict(color="red",width=5), opacity=0.5 ),               
#             )
            Rc, Zc = cycleRZ(phi)
            Rend, Zend = cycleRZ(phi) - arrowlen * eigvecs[:,i]
            Rarr1,Zarr1 = cycleRZ(phi) - 0.7 * arrowlen * eigvecs[:,i] + 0.3* arrowlen*np.array([[0.0, -1.0], [1.0, 0.0]])@eigvecs[:,i]
            Rarr2,Zarr2 = cycleRZ(phi) - 0.7 * arrowlen * eigvecs[:,i] - 0.3* arrowlen*np.array([[0.0, -1.0], [1.0, 0.0]])@eigvecs[:,i]
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rc*np.cos(phi), Rend*np.cos(phi)], 
                    y=[Rc*np.sin(phi), Rend*np.sin(phi)], 
                    z=[Zc, Zend], mode="lines", line=dict(color="red",width=5), opacity=0.5 ),               
            )
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rarr1*np.cos(phi), Rend*np.cos(phi)], 
                    y=[Rarr1*np.sin(phi), Rend*np.sin(phi)], 
                    z=[Zarr1, Zend], mode="lines", line=dict(color="red", width=5), opacity=0.5 ),               
            )
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rarr2*np.cos(phi), Rend*np.cos(phi)], 
                    y=[Rarr2*np.sin(phi), Rend*np.sin(phi)], 
                    z=[Zarr2, Zend], mode="lines", line=dict(color="red", width=5), opacity=0.5 ),               
            )
        elif  eigvals[i] < 1.0:
            Rc, Zc = cycleRZ(phi)
            Rend, Zend = cycleRZ(phi) + arrowlen * eigvecs[:,i]
            Rend2, Zend2 = cycleRZ(phi) + 0.7 * arrowlen * eigvecs[:,i]
            Rarr1,Zarr1 = cycleRZ(phi) + 1.0 * arrowlen * eigvecs[:,i] + 0.3* arrowlen*np.array([[0.0, -1.0], [1.0, 0.0]])@eigvecs[:,i]
            Rarr2,Zarr2 = cycleRZ(phi) + 1.0 * arrowlen * eigvecs[:,i] - 0.3* arrowlen*np.array([[0.0, -1.0], [1.0, 0.0]])@eigvecs[:,i]
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rc*np.cos(phi), Rend*np.cos(phi)], 
                    y=[Rc*np.sin(phi), Rend*np.sin(phi)], 
                    z=[Zc, Zend], mode="lines", line=dict(color="blue", width=5), opacity=0.5 ),               
            )
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rend2*np.cos(phi), Rarr1*np.cos(phi)], 
                    y=[Rend2*np.sin(phi), Rarr1*np.sin(phi)], 
                    z=[Zend2, Zarr1], mode="lines", line=dict(color="blue", width=5), opacity=0.5 ),               
            )
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rend2*np.cos(phi), Rarr2*np.cos(phi)], 
                    y=[Rend2*np.sin(phi), Rarr2*np.sin(phi)], 
                    z=[Zend2, Zarr2], mode="lines", line=dict(color="blue", width=5), opacity=0.5 ),               
            )
            Rc, Zc = cycleRZ(phi)
            Rend, Zend = cycleRZ(phi) - arrowlen * eigvecs[:,i]
            Rend2, Zend2 = cycleRZ(phi) - 0.70 * arrowlen * eigvecs[:,i]
            Rarr1,Zarr1 = cycleRZ(phi) - 1.0 * arrowlen * eigvecs[:,i] + 0.3* arrowlen*np.array([[0.0, -1.0], [1.0, 0.0]])@eigvecs[:,i]
            Rarr2,Zarr2 = cycleRZ(phi) - 1.0 * arrowlen * eigvecs[:,i] - 0.3* arrowlen*np.array([[0.0, -1.0], [1.0, 0.0]])@eigvecs[:,i]
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rc*np.cos(phi), Rend*np.cos(phi)], 
                    y=[Rc*np.sin(phi), Rend*np.sin(phi)], 
                    z=[Zc, Zend], mode="lines", line=dict(color="blue", width=5), opacity=0.5 ),               
            )
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rend2*np.cos(phi), Rarr1*np.cos(phi)], 
                    y=[Rend2*np.sin(phi), Rarr1*np.sin(phi)], 
                    z=[Zend2, Zarr1], mode="lines", line=dict(color="blue", width=5), opacity=0.5 ),               
            )
            fig.add_trace(    
                go.Scatter3d(
                    x=[Rend2*np.cos(phi), Rarr2*np.cos(phi)], 
                    y=[Rend2*np.sin(phi), Rarr2*np.sin(phi)], 
                    z=[Zend2, Zarr2], mode="lines", line=dict(color="blue", width=5), opacity=0.5 ),               
            )
            
            

# for phi in np.linspace(0.0, 2*m*np.pi , num=150):
#     draw_eigvec_at_phi(fig, phi, DPm_ivp, )

draw_eigvec_at_Phi(fig, np.linspace(0.0, 2*m*np.pi - 2.0/3*np.pi, num=50), DPm_ivp,)

cycleRZs = np.asarray( [cycleRZ(phi) for phi in Phi] )
fig.add_trace(    
    go.Scatter3d(
        x=cycleRZs[:,0]*np.cos(Phi), 
        y=cycleRZs[:,0]*np.sin(Phi), 
        z=cycleRZs[:,1], mode="lines", line=dict(color='green', width=5) ),               
)
            



fig.show()